In [3]:
!pip install webdriver-manager


  Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata (12 kB)
Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl (27 kB)


In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

chrome_service = ChromeService(ChromeDriverManager().install())
chrome_options = Options()
chrome_options.add_argument("--auto-open-devtools-for-tabs")
caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}


driver = webdriver.Chrome(service=chrome_service, options=chrome_options, desired_capabilities=caps)

# Open the HTML file
driver.get("https://www.naturalreaders.com/online/")  # Update this path


MaxRetryError: HTTPConnectionPool(host='0.0.0.0', port=4723): Max retries exceeded with url: /wd/hub/session (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8820f9d4c0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [24]:
test = driver.execute_script("var performance = window.performance || window.mozPerformance || window.msPerformance || window.webkitPerformance || {}; var network = performance.getEntries() || {}; return network;")

for item in test:
  print(item)

JavascriptException: Message: javascript error: circular reference
  (Session info: chrome=125.0.6422.112)
Stacktrace:
#0 0x563cfa328e3a <unknown>
#1 0x563cfa01245c <unknown>
#2 0x563cfa017b69 <unknown>
#3 0x563cfa019bdc <unknown>
#4 0x563cfa0a1289 <unknown>
#5 0x563cfa0814b2 <unknown>
#6 0x563cfa0a02cc <unknown>
#7 0x563cfa081253 <unknown>
#8 0x563cfa0511c7 <unknown>
#9 0x563cfa051b3e <unknown>
#10 0x563cfa2ef27b <unknown>
#11 0x563cfa2f3327 <unknown>
#12 0x563cfa2dbdae <unknown>
#13 0x563cfa2f3df2 <unknown>
#14 0x563cfa2c074f <unknown>
#15 0x563cfa318128 <unknown>
#16 0x563cfa3182fb <unknown>
#17 0x563cfa327f6c <unknown>
#18 0x7fb9164daea7 start_thread


In [11]:
driver.quit()


KeyboardInterrupt: 

To use `DesiredCapabilities` properly with a remote WebDriver, you need to set up a Selenium Grid or use a cloud-based Selenium service. For this example, I'll show you how to use a remote WebDriver to connect to a local Selenium Grid.

1. **Set up Selenium Grid locally:**
   - Start the Selenium Hub:
     ```sh
     java -jar selenium-server-standalone-x.xx.x.jar -role hub
     ```
   - Start a Node:
     ```sh
     java -jar selenium-server-standalone-x.xx.x.jar -role node -hub http://localhost:4444/grid/register
     ```

2. **Install the necessary packages if you haven't already:**
    ```sh
    pip install selenium webdriver-manager
    ```

3. **Use the following Python script:**
    ```python
    from selenium import webdriver
    from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
    import time
    import json

    # Define the remote WebDriver URL (Selenium Grid Hub)
    selenium_grid_url = "http://localhost:4444/wd/hub"

    # Setup desired capabilities
    caps = DesiredCapabilities.CHROME.copy()
    caps['goog:loggingPrefs'] = {'performance': 'ALL'}

    # Initialize the remote WebDriver
    driver = webdriver.Remote(
        command_executor=selenium_grid_url,
        desired_capabilities=caps
    )

    # Open the webpage
    driver.get("https://www.naturalreaders.com/online/")

    # Give some time for network activities to happen
    time.sleep(10)  # Adjust the sleep time as needed

    # Capture performance logs
    logs = driver.get_log('performance')

    # Process logs to extract URLs
    urls = []
    for entry in logs:
        log = json.loads(entry['message'])['message']
        if log['method'] == 'Network.requestWillBeSent':
            url = log['params']['request']['url']
            if 'blob:' in url:
                urls.append(url)

    driver.quit()

    # Print the extracted URLs
    for url in urls:
        print(url)
    ```

### Explanation

1. **Setup Selenium Grid Locally:**
   - Start the Selenium Hub and Node using the commands provided. Ensure you have the Selenium standalone server jar file (`selenium-server-standalone-x.xx.x.jar`).

2. **Define the Remote WebDriver URL:**
   - The `selenium_grid_url` is set to `http://localhost:4444/wd/hub`, which is the default address for a local Selenium Grid.

3. **Initialize the Remote WebDriver:**
   - Use `webdriver.Remote` to connect to the Selenium Grid Hub with the desired capabilities for performance logging.

4. **Open the Webpage:**
   - Navigate to the specified URL (`https://www.naturalreaders.com/online/`).

5. **Capture and Process Logs:**
   - Performance logs are captured and processed to find entries where the method is `Network.requestWillBeSent`.
   - URLs containing `blob:` are filtered and stored in the `urls` list.

6. **Output the URLs:**
   - The script prints the captured blob URLs.

### Additional Notes

- Adjust the `time.sleep(10)` as needed to allow enough time for network requests to be captured.
- Ensure you have the necessary permissions and the required packages installed.
- If you're using a cloud-based Selenium service, replace `selenium_grid_url` with the appropriate URL provided by the service.